In [ ]:
!pip install profanity-check

     |████████████████████████████████| 2.4MB 3.3MB/s 


In [ ]:
import json
import requests

def getQuill(text):
  url = "https://quillbot.p.rapidapi.com/paraphrase"

  payload = dict(
      text=text,
      numParaphrases=1,
      includeSegs=True,
      strength=3,
      autoflip=0.25,
  )

  headers = {
      'x-rapidapi-host': "quillbot.p.rapidapi.com",
      'x-rapidapi-key': "4e26127674msh44f405456fbb569p15cdd2jsn41c7e4a8bdd5",
      'content-type': "application/json",
      'accept': "application/json"
      }

  response = requests.request("POST", url, data=json.dumps(payload), headers=headers)
  return response.text

In [ ]:
import pandas as pd
import numpy as np
from profanity_check import predict, predict_prob
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/vzhou842/profanity-check/master/profanity_check/data/clean_data.csv")
df

,is_offensive,text
0,0,Then go to the village pump and suggest they c...
1,1,ANTI GREEK NATIONALIS -WIKIPEDIA \n\nHi Alexik...
2,1,Dis hoe wasnt dis violent on Lottery Ticket 😂😂
3,0,It is better for Atabay not helping the banned...
4,0,"""is in CamelCase. """"SiCKO"""" is not CamelCase,..."
...,...,...
184349,0,Template:uw-vandalism2 > | Talk
184350,1,"Regrets are for pussies. Shit happens, deal w..."
184351,0,Could this possibly be the origin of popular g...
184352,0,"""Your article submission has been declined, an..."


In [ ]:
off = df[df["is_offensive"] == 1]
X = off['text'].astype(str)
Y = off['is_offensive']
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.001, random_state = 42)

In [ ]:
x_test

13844     “@MutateRoos: You can be Seminoles but not red...
12752     "HOW DO YOU KNOW MY TWITTER PASSWORD" bitch pl...
179998    Refactoring\nDo not refactor my comments again...
65303               you're a cunt\ngo die in a fire kthx <3
65023                Obama ain't playin with these hoes... 
58621     Nuke Sweden! Go probe urself with an anal prob...
91730     "\n\nWhy should I give a fuck! LEAVE ME ALONE,...
172679    "\n\nJoe Johnston is honestly an idiot and has...
111853    I'm a big bitch and 4 is the GOAT  I'm too laz...
133154    I hate the white trash cops who raped and murd...
70734     Always good to have the ignorant and mentally ...
30868      bitches say "or nah" with everything I'm cute...
34383     First problem ParacusForward like most wiki ed...
87999     "\n\n Very serious stuff \n\nSherlock Holms, a...
103853    FUCK YOU DICK LICKS! \n\nYou guys will never b...
14680     laughable censorship \n\ni see that a post i d...
85746     "Wow! This is unbelievably sex

In [ ]:
baseline = len(x_test)
score =np.sum(predict(x_test)) 
baseline, score

(37, 32)

In [ ]:
def getQuillSet(x_test):
  res = []
  for x in x_test:
    print(x)
    quill = getQuill(x)
    quillList = json.loads(quill)
    for entry in quillList:
      print(entry["paraphrases"][0]["alt"])
      res.append(entry["paraphrases"][0]["alt"])
  return res
quillSet = getQuillSet(x_test)
quillSet

“@MutateRoos: You can be Seminoles but not redskins? K”Seminole tribe got an agreement with them..fsu pay them to use the name
"@mutateroos: can you be a seminol but not a redskin?
k'seminole tribe got an deal with them to use the name.fu pay them
"HOW DO YOU KNOW MY TWITTER PASSWORD" bitch please
"How do you know my secret Facebook" slut open
Refactoring
Do not refactor my comments again. You have been in dispute with me in the past, and perhaps you don't like me, but you have no business mucking about with what I have ot say. If you object to any of my comments feel free to bring them up at the appropriate noticeboard, but do not fuck with them again. Thanks.
Refactoring Should not re-engineer my remarks.
In the past, you've been in disagreement with me, and maybe you don't trust me, but you don't have much to do with what I've said.
 If you object to any of my comments, please put them up on the correct noticeboard, just don't screw them again.
thank you.
you're a cunt
go die in a f

['"@mutateroos: can you be a seminol but not a redskin?',
 "k'seminole tribe got an deal with them to use the name.fu pay them",
 '"How do you know my secret Facebook" slut open',
 'Refactoring Should not re-engineer my remarks.',
 "In the past, you've been in disagreement with me, and maybe you don't trust me, but you don't have much to do with what I've said.",
 " If you object to any of my comments, please put them up on the correct noticeboard, just don't screw them again.",
 'thank you.',
 'you are a slut go died in a kthx explosion <3',
 ' For these hoes, Obama is not running...',
 'nuke sweden to you!',
 'go try an anal probe on yourself.',
 'f * blackhead.',
 '"Why would i give a goddamn fuck! leave me alone, you stupid admins!!!',
 '"joe johnston is really an fool who has been blabbing about jp4 for almost 10 years. the jp series is gone. if there is only one, the latest film does not have the jp4 title. it\'s very confusing and irritating how much knowledge contradicts one an

In [ ]:
quillScore = np.sum(predict(quillSet))
quillScore

31

In [ ]:
len(quillSet)

89

In [ ]:
scoreProb = np.mean(predict_prob(x_test)) 
quillScoreProb = np.mean(predict_prob(quillSet))
scoreProb, quillScoreProb

(0.8052124016534462, 0.37935217653388065)